In [1]:
import numpy as np
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [2]:
from __future__ import print_function

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

from keras import regularizers
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras import backend as K
from keras import metrics

def build_model(data):
    batch_size = 20
    original_dim=data.shape[1]
    latent_dim = 256

    epochs = 100
    epsilon_std = 1.0


    x = Input(shape=(original_dim,))
    h = Dense(2048, activation='tanh',
                    activity_regularizer=regularizers.l1(1*10e-5))(x)
    h = Dense(1024, activation='tanh',
                    activity_regularizer=regularizers.l1(1*10e-5))(h)


    z_mean = Dense(latent_dim)(h)
    z_log_var = Dense(latent_dim)(h)


    def sampling(args):
        z_mean, z_log_var = args
        epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0.,
                                  stddev=epsilon_std)
        return z_mean + K.exp(z_log_var / 2) * epsilon

    # note that "output_shape" isn't necessary with the TensorFlow backend
    z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

    # we instantiate these layers separately so as to reuse them later

    h_decoded = Dense(1024, activation='tanh')(z)
    h_decoded = Dense(2048, activation='tanh')(h_decoded)
    x_decoded_mean = Dense(original_dim, activation='tanh')(h_decoded)

    # instantiate VAE model
    vae = Model(x, x_decoded_mean)

    # Compute VAE loss
    xent_loss = original_dim * metrics.binary_crossentropy(x,x_decoded_mean )
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    vae_loss = K.mean(xent_loss + kl_loss)

    vae.add_loss(vae_loss)
    from keras import optimizers
    rmsprop=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=None, decay=0.0)
    vae.compile(optimizer=rmsprop)
    vae.summary()


    vae.fit(data,
            shuffle=True,
            epochs=epochs,
            batch_size=batch_size)

    encoder = Model(x, z)




    return encoder

Using TensorFlow backend.


In [3]:
from nilearn.decomposition import CanICA
def prepare_data(func_filenames):
    canica = CanICA(memory="nilearn_cache", memory_level=2,
                    threshold=3., verbose=10, random_state=0, 
                    mask='/home/share/TmpData/Qinglin/ADHD200_Athena_preproc_flirtfix/ADHD200_mask_152_4mm.nii.gz')
    data=canica.prepare_data(func_filenames)
    return data

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
from nilearn.connectome import ConnectivityMeasure

def corr(all_time_series):
    connectivity_biomarkers = {}
    conn_measure = ConnectivityMeasure(kind='correlation', vectorize=True)
    connectivity_biomarkers = conn_measure.fit_transform(all_time_series)
    return connectivity_biomarkers

In [5]:
from nilearn.connectome import ConnectivityMeasure

def corr_tan(all_time_series):
    connectivity_biomarkers = {}
    tangent_measure = ConnectivityMeasure(kind='tangent', vectorize=True)
    connectivity_biomarkers = tangent_measure.fit_transform(all_time_series)
    return connectivity_biomarkers


In [6]:
import numpy as np
X=np.load('/home/share/TmpData/Qinglin/ABIDE/X.npy')
Y=np.load('/home/share/TmpData/Qinglin/ABIDE/Y.npy')
model=build_model(X)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28546)        0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 2048)         58464256    input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 1024)         2098176     dense_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 256)          262400      dense_2[0][0]                    
__________________________________________________________________________________________________
dense_4 (D

17200/17200 [==============================] - 57s 3ms/step - loss: -285299.7100
Epoch 64/100
17200/17200 [==============================] - 57s 3ms/step - loss: -285462.2343
Epoch 65/100
17200/17200 [==============================] - 57s 3ms/step - loss: -285831.0069
Epoch 66/100
17200/17200 [==============================] - 56s 3ms/step - loss: -286197.3811
Epoch 67/100
17200/17200 [==============================] - 56s 3ms/step - loss: -286375.4339
Epoch 68/100
17200/17200 [==============================] - 56s 3ms/step - loss: -286546.1025
Epoch 69/100
17200/17200 [==============================] - 56s 3ms/step - loss: -286773.1125
Epoch 70/100
17200/17200 [==============================] - 56s 3ms/step - loss: -287070.2298
Epoch 71/100
17200/17200 [==============================] - 57s 3ms/step - loss: -287398.2424
Epoch 72/100
17200/17200 [==============================] - 57s 3ms/step - loss: -287483.6824
Epoch 73/100
17200/17200 [==============================] - 57s 3ms/step 

In [7]:


from nilearn.datasets import fetch_abide_pcp

# We specify the site and number of subjects we want to download
abide = fetch_abide_pcp(derivatives=['func_preproc'], data_dir='/home/share/TmpData/Qinglin/nilearn_data/')

# We look at the available data in this dataset
print(abide.keys())


func_filenames = abide.func_preproc  # list of 4D nifti files for each subject

from nilearn._utils.niimg_conversions import _resolve_globbing
imgs = _resolve_globbing(func_filenames)

mask_img ='/home/share/TmpData/Qinglin/ADHD200_Athena_preproc_flirtfix/ADHD200_mask_152_4mm.nii.gz'

from nilearn.input_data import NiftiMasker
masker = NiftiMasker(mask_img=mask_img, 
                     standardize=True,
                     detrend=1,
                     smoothing_fwhm=6.,
                     memory="/storage/nilearn_cache", 
                     memory_level=2)
fmri_masked = masker.fit()

from nilearn.decomposition.base import _mask_and_reduce_single
all_time_series=[]
for img in imgs:
    print(img)
    data = _mask_and_reduce_single(
        masker, img, confound=None,
        reduction_ratio=1,
        random_state=0,
        memory_level=3)
    time_series=model.predict(data,
        batch_size=20)
    all_time_series.append(time_series)

D=all_time_series    

/usr/local/lib/python3.5/dist-packages/numpy/lib/npyio.py:2278: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  output = genfromtxt(fname, **kwargs)


dict_keys(['phenotypic', 'description', 'func_preproc'])


/usr/local/lib/python3.5/dist-packages/nilearn/_utils/cache_mixin.py:84: DeprecationWarning: The 'cachedir' parameter has been deprecated in version 0.12 and will be removed in version 0.14.
You provided "cachedir='/storage/nilearn_cache'", use "location='/storage/nilearn_cache'" instead.
  memory = Memory(cachedir=cache_dir, verbose=verbose)
/usr/local/lib/python3.5/dist-packages/nilearn/_utils/cache_mixin.py:232: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if (memory.cachedir is None and memory_level is not None


/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/Pitt_0050003_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/Pitt_0050004_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/Pitt_0050005_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/Pitt_0050006_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/Pitt_0050007_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/Pitt_0050008_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/Pitt_0050010_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/Pitt_0050011_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/Pitt_0050012_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac

/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/OHSU_0050143_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/OHSU_0050144_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/OHSU_0050145_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/OHSU_0050146_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/OHSU_0050147_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/OHSU_0050148_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/OHSU_0050149_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/OHSU_0050150_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/OHSU_0050152_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac

...

/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051017_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051018_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051019_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051020_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051021_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051023_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051024_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051025_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051026_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_n

/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051102_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051103_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051104_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051105_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051106_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051107_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051109_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051110_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051111_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_n

/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/UCLA_1_0051207_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/UCLA_1_0051208_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/UCLA_1_0051210_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/UCLA_1_0051211_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/UCLA_1_0051212_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/UCLA_1_0051214_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/UCLA_1_0051215_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/UCLA_1_0051216_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/UCLA_1_0051217_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_d

/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/UCLA_2_0051313_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/UCLA_2_0051315_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/MaxMun_a_0051318_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/MaxMun_a_0051319_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/MaxMun_a_0051320_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/MaxMun_a_0051321_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/MaxMun_b_0051322_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/MaxMun_b_0051323_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/MaxMun_b_0051325_func_preproc.nii.gz
/home/share/TmpData/Qin

/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/SBL_0051574_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/SBL_0051576_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/SBL_0051577_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/SBL_0051578_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/SBL_0051579_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/SBL_0051580_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/SBL_0051582_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/SBL_0051583_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/SBL_0051584_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_n

In [8]:
for d in D:
    pass
print(d.shape)    


(116, 256)


In [9]:
from joblib import Parallel, delayed

corrD=Parallel(n_jobs=8)(delayed(corr_tan)(d) for d in D)

ValueError: Each subject must be 2D numpy.ndarray.
 Youprovided arrays of dimensions [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier

names = ["Nearest Neighbors", "RBF SVM", 
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA","XGBoost","Bagging","GTBoosting"]

classifiers = [
    KNeighborsClassifier(3),
#         SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
#         GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=10),
    RandomForestClassifier(max_depth=10, n_estimators=100, max_features=1),
    MLPClassifier(),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(),
    XGBClassifier(),
    BaggingClassifier(KNeighborsClassifier(),max_samples=0.5, max_features=0.5),
    GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0)]

from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

from sklearn.model_selection import cross_val_score
mean_scores = []

for name, clf in zip(names, classifiers):
    cv_scores = cross_val_score(clf,
                                corrD,
                                y=Y,
                                cv=cv,
                                groups=Y,
                                scoring='accuracy',
                                )
    mean_scores.append(cv_scores.mean())

In [ ]:
from nilearn.plotting import show
import matplotlib.pylab as plt

plt.figure(figsize=(6, 4))
positions = np.arange(len(names)) * .1 + .1
plt.barh(positions, mean_scores, align='center', height=.05)
yticks = [name.replace(' ', '\n') for name in names]
plt.yticks(positions, yticks)
plt.xlabel('Classification accuracy')
plt.grid(True)
plt.tight_layout()

show()